# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


<div class="alert alert-info">
Никита, привет! Рад познакомиться :).
Большое спасибо за подсказки и за замечания. Я закомментировал некоторые поля, чтобы ты видел старый код и оценил мощь нового :))
 
Постарался исправить все недочёты, которые ты указал. Приятного просмотра!

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [ ]:
import pandas as pd
from pymystem3 import Mystem 
m = Mystem()
from collections import Counter 



df = pd.read_csv('/datasets/data.csv')

In [ ]:
df.columns #Проверяем корректность названия столбцов. Пробелов и ошибок не обнаружено

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [ ]:
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
df.info() #Получаем общую информацию о таблице, типах данных
df.isnull().sum() #Вычисляем количество пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [ ]:
days_employed_real_numb = (70 - 18) * 247
print(days_employed_real_numb)

12844


Где:
    70 - возраст, до которого человек работает (с запасом к пенисонному)
    18 - возраст зачисления на работу 
    247 - среднее количество рабочих дней в году
Таким образом, мы получили более-менее реалистичную цифру. Однако в нашей таблице мы увидели значения, которые намного превышают это число.


**Вывод** 



В прочитанном нами файле обнаружены следующие недочёты:
1. В столбце 'days_employed' во множестве случаев представлено число отработанных дней со знаком минус. Также имеется артефакт с данными отработанных дней у людей, которые старше 50 лет.
2. В столбце 'education' сведения об образовании заполнены везде по-разному: заглавными, строчными, заглавными и строчными буквами.
3. В столбце 'purpose' имеются одинаковые цели получения кредита, которые написанные разными словами

Вызвав метод info() , были обнаружены пропуски в столбцах:
1. days_employed (NaN) -float64
2. total_income (NaN) -float64

Вызвав метод describe() , мы обнаружили аномалии в столбце days_employed.Показатель mean гласит, что якобы в среднем люди работали по 66 000 дней, что никак невозможно. Относительно реальное количество рабочих дней составляет порядка 13 тысяч. В следующее наше действие - более детально изучить эти данные и принять решение каким способом их заменить.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [ ]:
df['days_employed'] = df['days_employed'].abs()
display(df.head(10))


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [ ]:
#df.loc[df.isnull().any(axis=1)] #Запрашиваем строки с пропущенными значениями

In [ ]:
days_empl_median = df['days_employed'].median()

In [ ]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [ ]:
#df['total_income'] > days_employed_real_numb
df.loc[(df['days_employed'] > days_employed_real_numb)].count()

children            3504
days_employed       3504
dob_years           3504
education           3504
education_id        3504
family_status       3504
family_status_id    3504
gender              3504
income_type         3504
debt                3504
total_income        3504
purpose             3504
dtype: int64

In [ ]:
df['days_employed'].mean()

66914.72890682236

In [ ]:
df.loc[(df['days_employed'] > days_empl_median)].count()

children            9675
days_employed       9675
dob_years           9675
education           9675
education_id        9675
family_status       9675
family_status_id    9675
gender              9675
income_type         9675
debt                9675
total_income        9675
purpose             9675
dtype: int64

In [ ]:
df.pivot_table(index='income_type', values=['days_employed', 'dob_years'], aggfunc='median')

,days_employed,dob_years
income_type,,
безработный,366413.652744,38.0
в декрете,3296.759962,39.0
госслужащий,2689.368353,40.0
компаньон,1547.382223,39.0
пенсионер,365213.306266,60.0
предприниматель,520.848083,42.5
сотрудник,1574.202821,39.0
студент,578.751554,22.0


При выгрузке некоторых данных явно произошла какая-то ошибка, либо данные в систему были внесены некорректно. Человек с типом занятости "Сотрудник" и медианным возрастом 39 лет отработал всего 1574 дня, что составляет порядка 6 лет стажа, то есть получается, что сотрудник якобы устроился на работу в 33 года, ну или до этого примерно 10 лет работал неофициально. Допустим, это возможно, это значение мы оставим.
Люди с типом занятости "Пенсионер" и "Безработный" выглядят абсолютно неправдоподобно. Была допущена ошибка. Вполне возможно, что сотрудник внёс данные об отработанных часах, в таком случае, это похоже на правду для пенсионера, но никак не похоже на безработного (если ещё учесть, что медианный возраст 38 лет)
Данные по остальным группам выглядят правдоподобно.

Что делаем:
1. В пропусках для категорий "Сотрудник", "В декрете", "Студент", "Госслужащий", "Предприниматель", "Компаньон" заменяем данные по медиане этих категорий
2. В пропусках для категории "Пенсионер" заменяем данные по медиане, разделенной на 24 (ч)
3. В пропусках для категории "Безработный" заменяем данные по общей медиане. Такое решение я принял в связи с тем, что какое-то минимальное количество рабочих дней так или иначе было отработано этими людьми, поэтому я взял самое минимальное медианное число из всех представленных.


In [ ]:
especially_group = 'сотрудник', 'компаньон', 'госслужащий','предприниматель', 'студент', 'в декрете'

In [ ]:
print(especially_group)

('сотрудник', 'компаньон', 'госслужащий', 'предприниматель', 'студент', 'в декрете')


In [ ]:
df.loc[df['income_type']=='пенсионер', 'days_employed'] /= 24

pens_median = df.loc[(df['income_type'] =='пенсионер') & (df['days_employed'])].median()

print(pens_median)

children                 0.000000
days_employed        15217.221094
dob_years               60.000000
education_id             1.000000
family_status_id         0.000000
debt                     0.000000
total_income        118514.486412
dtype: float64


In [ ]:
df.loc[(df['days_employed'].isna())&(df['income_type']=='пенсионер'), 'days_employed' ] = 15217

In [ ]:
df.loc[(df['income_type'] =='пенсионер')].isna().sum()

children              0
days_employed         0
dob_years             0
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income        413
purpose               0
dtype: int64

In [ ]:
df.loc[(df['income_type'] =='безработный')].count()

children            2
days_employed       2
dob_years           2
education           2
education_id        2
family_status       2
family_status_id    2
gender              2
income_type         2
debt                2
total_income        2
purpose             2
dtype: int64

In [ ]:
df.loc[df['income_type']=='безработный', 'days_employed'] = df['days_employed'].median()

In [ ]:
df.loc[(df['income_type'] =='безработный')]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,2269.733207,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,2269.733207,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [ ]:
def change_nan(data, category, value):
  for type in especially_group:
    data.loc[(data[value].isna())&(data[category]==type), value ] =\
data.loc[data[category]==type, value].median()

In [ ]:
change_nan(df, 'income_type', 'days_employed')

In [ ]:
df.info()

#df.loc[(df['total_income'].isnull())&(df['income_type'].count()

#df.pivot_table(index='income_type', values='days_employed', aggfunc='mean')
                                      

#df.loc[df.isnull().any(axis=1)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [ ]:
df['days_employed'].isna().sum()

0

In [ ]:
def change_nan(dataset, category, value):
    for type in dataset[category].unique():
        dataset.loc[(dataset[value].isna())&(dataset[category]==type), value ] =\
dataset.loc[dataset[category]==type, value].median()


<div class="alert alert-success">
<h1>Комментарий ревьюера v2 <a class="tocSkip"></a></h1>
Молодец, что поправил замечание:)</div>

In [ ]:
change_nan(df, 'income_type', 'total_income')

In [ ]:
df['total_income'].isna().sum()

0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**
Обнаруженные аномальные значения в столбце days_employed были вычислены и заменены, также мы обработали все пропуски.
С помощью функции были обработаны все пропуски в столбце total_income.

In [ ]:
#df.dropna(subset=['days_employed', 'total_income'], inplace=True) #удаляем все строки, где есть пропущенные значения

In [ ]:
#df.info() #проверяем удалились ли дубликаты

### Замена типа данных

In [ ]:
df['days_employed'] = df['days_employed'].astype(int) # заменяем тип данных float64 на int в столбце days_employed
df['total_income'] = df['total_income'].astype(int)# заменяем тип данных float64 на int в столбце total_income
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**
Заменены значения на целочисленные

### Обработка дубликатов

In [ ]:
df.duplicated().sum() #вычисляем сумму дубликатов в df

54

In [ ]:
df.loc[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
4182,1,1574,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
7808,0,15217,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8583,0,15217,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
9238,2,1574,34,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для сдачи
9528,0,15217,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью
9627,0,15217,56,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции со своей недвижимостью
10462,0,15217,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,покупка коммерческой недвижимости


In [ ]:
df['education'].unique() #посмотрим на количество уникальных значений в столбце education

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [ ]:
df['education'] = df['education'].str.lower() #сделаем все буквы строчными, чтобы избавиться от дубликатов

df['education'].unique() #проверяем результат замены

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [ ]:
df.loc[df.duplicated()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,1574,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,15217,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,15217,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,15217,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,15217,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием
21132,0,1574,47,среднее,1,женат / замужем,0,F,сотрудник,0,142594,ремонт жилью
21281,1,1574,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости


In [ ]:
df['purpose'].value_counts() #проверяем наличие дубликатов в столбце purpose

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**
Ручным способом были обнаружены дубликаты в столбце purpose. Похожие слова, измененный порядок слов.

### Лемматизация

In [ ]:

df['purpose_new'] = df['purpose'].apply(m.lemmatize) #создаем новый столбец с лемматизированными словами


print(Counter(df['purpose_new'].sum())) #получаем часто используемые цели кредита. Выделили 4 категории: недвижимость, образование, автомобиль, свадьба




Counter({' ': 33677, '\n': 21525, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'подержанный': 489, 'подержать': 479, 'приобретение': 462, 'профильный': 436})



<div class="alert alert-info">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Все импорты лучше располагать в начале проекта</div>

In [ ]:
def change_purpose(purpose):
    if 'жилье'  in purpose:
        return 'недвижимость'
    if 'автомобиль' in purpose:
        return 'автомобиль'
    if 'образование' in purpose:
        return 'образование'
    if 'свадьба' in purpose:
        return 'свадьба'
    if 'недвижимость' in purpose:
        return 'недвижимость'

df['purpose_aggr'] = df['purpose_new'].apply(change_purpose) #создали новый столбец с измененной целью кредита
print(df.tail(10))

       children  days_employed  dob_years       education  education_id  \
21515         1            467         28         среднее             1   
21516         0            914         42          высшее             0   
21517         0            404         42          высшее             0   
21518         0          15583         59         среднее             1   
21519         1           2351         37  ученая степень             4   
21520         1           4529         43         среднее             1   
21521         0          14330         67         среднее             1   
21522         1           2113         38         среднее             1   
21523         3           3112         38         среднее             1   
21524         2           1984         40         среднее             1   

          family_status  family_status_id gender income_type  debt  \
21515   женат / замужем                 0      F   сотрудник     1   
21516   женат / замужем           

**Вывод**

С помощью лемматизации выделили 4 основные цели кредита

### Категоризация данных

In [ ]:
#разобьём наших заемщиков на категории по уровню ЗП

def category_income(total_income):
    if total_income < 30000:
        return('низкий')
    if total_income < 70000:
        return('ниже среднего')
    if total_income < 120000:
        return('средний')
    if total_income > 120000:
        return('высокий')
#print(category_income(1000000)) # проверка работоспособности функции

df['category_income'] = df['total_income'].apply(category_income) #добавляем новый столбец с уровнем дохода, согласно нашему правилу

print(df)

       children  days_employed  dob_years education  education_id  \
0             1           8437         42    высшее             0   
1             1           4024         36   среднее             1   
2             0           5623         33   среднее             1   
3             3           4124         32   среднее             1   
4             0          14177         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1           4529         43   среднее             1   
21521         0          14330         67   среднее             1   
21522         1           2113         38   среднее             1   
21523         3           3112         38   среднее             1   
21524         2           1984         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

**Вывод**

Провели категоризацию по уровню дохода, чтобы решить одну из поставленных задач.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
#childfree = df.loc[(df['children']==0)&(df['debt']==0), 'debt'].count()
#print(childfree)

#parents = df.loc[(df['children']>0)&(df['debt']==0), 'debt'].count()

#debt_parents = df.loc[(df['children']>0)&(df['debt']==1), 'debt'].count()

#debt_childfree = df.loc[(df['children']==0)&(df['debt']==1), 'debt'].count()

#total = childfree + parents + debt_parents + debt_childfree
#print(total)
#part_childfree_debt = debt_childfree / childfree
#part_parents_debt = debt_parents / parents


#print('Количество бездетных не имеющих задолженности:', childfree)
#print('Количество бездетных с задолженностью:', debt_childfree,', ' 'Доля от числа бездетных: {:.1%}'.format(part_childfree_debt))
#print('Кол-во родителей не имеющих задолженности:', parents)
#print('Количество родителей с задолженностью:', debt_parents, ', ' 'Доля от числа родителей: {:.1%}'.format(part_parents_debt))

13086
21478
Количество бездетных не имеющих задолженности: 13086
Количество бездетных с задолженностью: 1063 , Доля от числа бездетных: 8.1%
Кол-во родителей не имеющих задолженности: 6652
Количество родителей с задолженностью: 677 , Доля от числа родителей: 10.2%


In [ ]:
df.pivot_table(index='debt', columns='children', aggfunc='count')


category_income                                                   \
children             -1        0       1       2      3     4    5     20   
debt                                                                        
0                   46.0  13086.0  4374.0  1861.0  303.0  37.0  9.0  68.0   
1                    1.0   1063.0   444.0   194.0   27.0   4.0  NaN   8.0   

         days_employed           ... purpose_new       total_income           \
children           -1        0   ...          5     20          -1        0    
debt                             ...                                           
0                 46.0  13086.0  ...         9.0  68.0         46.0  13086.0   
1                  1.0   1063.0  ...         NaN   8.0          1.0   1063.0   

                                                  
children      1       2      3     4    5     20  
debt                                              
0         4374.0  1861.0  303.0  37.0  9.0  68.0  
1          444.0   194.0   27.0   4.0  NaN   8.0  

[2 rows x 104 columns]

In [ ]:
df.groupby('children')['debt'].mean()

children
-1     0.021277
 0     0.075129
 1     0.092154
 2     0.094404
 3     0.081818
 4     0.097561
 5     0.000000
 20    0.105263
Name: debt, dtype: float64

**Вывод**

Если судить по цифрам, то есть незначительная зависимость между наличием детей и возвратом кредита в срок. Но разница небольшая: порядка 2%. Если ещё учесть тот факт, что родителям нужно явно больше средств, чем бездетным, то эту разницу между этими категорями можно не принимать во внимание. Скажем, если бы разница была от 10% и более, то да, могли бы увеличить ставку по кредиту для родителей, но вместе с этим пришлось бы увеличить срок погашения кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
print(df.groupby('family_status')['debt'].value_counts())

married = df[df['family_status'] == 'женат / замужем']['family_status'].count()
debt_married = df.loc[(df['family_status']=='женат / замужем')&(df['debt']==1), 'debt'].count()
debt_to_married = debt_married / married

unmarried = df[df['family_status'] == 'Не женат / не замужем']['family_status'].count()
debt_unmarried = df.loc[(df['family_status']=='Не женат / не замужем')&(df['debt']==1), 'debt'].count()
debt_to_unmarried = debt_unmarried / unmarried

wid = df[df['family_status'] == 'вдовец / вдова']['family_status'].count()
debt_wid = df.loc[(df['family_status']=='вдовец / вдова')&(df['debt']==1), 'debt'].count()
debt_to_wed = debt_wid / wid

roommates = df[df['family_status'] == 'гражданский брак']['family_status'].count()
debt_roommates = df.loc[(df['family_status']=='гражданский брак')&(df['debt']==1), 'debt'].count()
debt_to_roommates = debt_roommates / roommates

print('Доля должников среди женатых составляет: {:.1%}'.format(debt_to_married))
print('Доля должников среди неженатых / незамужних составляет: {:.1%}'.format(debt_to_unmarried))
print('Доля должников среди вдов  составляет: {:.1%}'.format(debt_to_wed))
print('Доля должников среди сожителей в гражданском браке составляет: {:.1%}'.format(debt_to_roommates))

family_status          debt
Не женат / не замужем  0        2271
                       1         254
в разводе              0        1007
                       1          76
вдовец / вдова         0         809
                       1          56
гражданский брак       0        3396
                       1         339
женат / замужем        0       10297
                       1         846
Name: debt, dtype: int64
Доля должников среди женатых составляет: 7.6%
Доля должников среди неженатых / незамужних составляет: 10.1%
Доля должников среди вдов  составляет: 6.5%
Доля должников среди сожителей в гражданском браке составляет: 9.1%


**Вывод**

Есть определенная зависимость. Чем старше человек, тем меньше вероятность появления задолженности. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
print(df.groupby('category_income')['debt'].value_counts())

total_hight = df[df['category_income'] == 'высокий']['category_income'].count()
debt_hight = df.loc[(df['category_income']=='высокий')&(df['debt']==1), 'debt'].count()
debt_to_hight = debt_hight / total_hight

total_middle = df[df['category_income'] == 'средний']['category_income'].count()
debt_middle = df.loc[(df['category_income']=='средний')&(df['debt']==1), 'debt'].count()
debt_to_middle = debt_middle / total_middle

total_lower_mid = df[df['category_income'] == 'ниже среднего']['category_income'].count()
debt_lower_mid = df.loc[(df['category_income']=='ниже среднего')&(df['debt']==1), 'debt'].count()
debt_to_lower_mid = debt_lower_mid / total_lower_mid

total_low = df[df['category_income'] == 'низкий']['category_income'].count()
debt_low = df.loc[(df['category_income']=='низкий')&(df['debt']==1), 'debt'].count()
debt_to_low = debt_low / total_low 

print('Доля должников среди обеспеченных составляет: {:.1%}'.format(debt_to_hight))
print('Доля должников среди среднего класса составляет: {:.1%}'.format(debt_to_middle))
print('Доля должников среди ниже среднего класса  составляет: {:.1%}'.format(debt_to_lower_mid))
print('Доля должников среди низкого класса составляет: {:.1%}'.format(debt_to_low))


category_income  debt
высокий          0       11486
                 1        1020
ниже среднего    0        1353
                 1          99
низкий           0          20
                 1           2
средний          0        4921
                 1         450
Name: debt, dtype: int64
Доля должников среди обеспеченных составляет: 8.2%
Доля должников среди среднего класса составляет: 8.4%
Доля должников среди ниже среднего класса  составляет: 6.8%
Доля должников среди низкого класса составляет: 9.1%


In [ ]:
df.groupby('category_income')['debt'].mean()

category_income
высокий          0.080956
ниже среднего    0.068182
низкий           0.090909
средний          0.083852
Name: debt, dtype: float64

**Вывод**

Те, кто зарабатывают меньше всего имеют более высокий процент просрочек. Самые расчётливые - люди ниже среднего класса

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
print(df.groupby('purpose_aggr')['debt'].value_counts())

total_cars = df[df['purpose_aggr'] == 'автомобиль']['purpose_aggr'].count()
debt_cars = df.loc[(df['purpose_aggr']=='автомобиль')&(df['debt']==1), 'debt'].count()
debt_to_cars = debt_cars / total_cars

total_flats = df[df['purpose_aggr'] == 'недвижимость']['purpose_aggr'].count()
debt_flats = df.loc[(df['purpose_aggr']=='недвижимость')&(df['debt']==1), 'debt'].count()
debt_to_flats = debt_flats / total_flats

total_lower_ed = df[df['purpose_aggr'] == 'образование']['purpose_aggr'].count()
debt_lower_ed = df.loc[(df['purpose_aggr']=='образование')&(df['debt']==1), 'debt'].count()
debt_to_lower_ed = debt_lower_ed / total_lower_ed

total_wedding = df[df['purpose_aggr'] == 'свадьба']['purpose_aggr'].count()
debt_wedding = df.loc[(df['purpose_aggr']=='свадьба')&(df['debt']==1), 'debt'].count()
debt_to_wedding = debt_wedding / total_wedding 

print('Доля должников среди владельцев авто составляет: {:.1%}'.format(debt_to_cars))
print('Доля должников среди владельцев недвижимости составляет: {:.1%}'.format(debt_to_flats))
print('Доля должников среди обучающихся  составляет: {:.1%}'.format(debt_to_lower_ed))
print('Доля должников среди женатых составляет: {:.1%}'.format(debt_to_wedding))


purpose_aggr  debt
автомобиль    0       3530
              1        367
недвижимость  0       9043
              1        715
образование   0       3266
              1        331
свадьба       0       1941
              1        158
Name: debt, dtype: int64
Доля должников среди владельцев авто составляет: 9.4%
Доля должников среди владельцев недвижимости составляет: 7.3%
Доля должников среди обучающихся  составляет: 9.2%
Доля должников среди женатых составляет: 7.5%


In [ ]:
df.groupby('purpose_aggr')['debt'].mean()

purpose_aggr
автомобиль      0.093395
недвижимость    0.072140
образование     0.091994
свадьба         0.079216
Name: debt, dtype: float64


<div class="alert alert-warning">
<h1>Комментарий ревьюера <a class="tocSkip"></a></h1>
Очень много кода, тут можно через groupby:
    df.groupby('purpose_aggr')['debt'].mean() --- процент должников
    
 </div>

**Вывод**

Получается, что самые исправные плательщики - владельцы недвижимости. Можно предположить, что эти люди как минимум погашают задолженность вдвоём, а также у них есть стабильная работа в отличие от студентов, например. Следующая категория - молодожены. Вероятно, сумма платежа небольшая, плюс какая-то часть средств может быть подарена родственниками, поэтому нет больших просрочек. Второе место занимают студенты. Можем предположить, что у них ещё нет работы или низкооплачиваемая работа и денег едва хватает на бытовые расходы. На первом месте владельцы авто, можем предположить, что эта категория людей довольно много денег тратит на обслуживание автомобиля (особенно, если это б/у), страховку и прочие расходы.

## Шаг 4. Общий вывод

На основании проанализированных данных можно выделить самые предпочтительные категории людей и цели кредита:
1. Молодоженам на свадьбу
2. Людям, которые ближе к среднему и обеспеченному классу
3. Вдовцы / вдовы
4. Недвижимость

На наличие / отсутствие детей можно не обращать внимание - разница между задолженностью небольшая. Самые небезопасные цели кредита - это автомобиль и обучение. Также высока доля должников среди низкооплачиваемых людей
